In [276]:
# Initialize Otter
import otter
grader = otter.Notebook("hw02B.ipynb")

# Homework 2B: Food Safety (Continued)

## Cleaning and Exploring Data with `pandas`

## Due Date: Thursday, September 19, 11:59 PM
You must submit this assignment to Gradescope by the on-time deadline, Thursday, September 19, 11:59 PM. Please read the syllabus for the Slip Day policy. No late submissions beyond what is outlined in the Slip Day policy will be accepted. **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance (TAs need to sleep, after all!).

Please read the instructions carefully when you are submitting your work to Gradescope.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** below.

**Collaborators**: *list collaborators here*


## This Assignment

In this homework, we will continue our exploration of restaurant food safety scores for restaurants in San Francisco. The main goal for this assignment is to focus more on the analysis of the dataset, building on the data cleaning we have done earlier in HW 2A. 


After this homework, you should be comfortable with:
* Reading `pandas` documentation and using `pandas` methods
* Working with data at different levels of granularity
* Using `.groupby` with different aggregation functions
* Chaining different `pandas` functions and methods to find answers to exploratory questions


## Score Breakdown 
Question | Manual | Points
--- | --- | ---
1a | no | 2
1b | no | 3
1c | no | 3
2a | no | 2
2b | no | 3
2c | no | 1
2d | no | 1
3a | no | 1
3b | no | 2
3c | yes | 3
3d | no | 3
3e | yes | 3
4 | no | 3
Total | 2 | 30


## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run code, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `pandas` and `NumPy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.

### Debugging Guide
If you run into any technical issues, we highly recommend checking out the [Data 100 Debugging Guide](https://ds100.org/debugging-guide/). In this guide, you can find general questions about Jupyter notebooks / Datahub, Gradescope, and common pandas errors.

In [277]:
import numpy as np
import pandas as pd

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

In HW 2A, we took you through the entire process of reading data from a file to perform some exploration of the data. Here, we again load the dataset that we will be using in HW 2B along with some of the columns we had added in HW 2A. For any additional context regarding the dataset, feel free to revisit HW 2A.

In [278]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand=True)[0].astype(int) 

# This code is essential for the autograder to function properly. Do not edit.
ins_test = ins

<br/><br/>

---


# Question 1: Inspecting the Inspections


## Question 1a

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [279]:
ins.head(5)

,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `df.rename()` to be useful!

In [280]:
ins_score_by_type = ( 
    ins
    .groupby("type")
    .max("score")
    .rename(columns={"score":"max_score"})
    .drop(columns=["bid"])
)
ins_score_by_type

,max_score
type,
Administrative or Document Review,-1
Community Health Assessment,-1
Complaint,-1
Complaint Reinspection/Followup,-1
Foodborne Illness Investigation,-1
Multi-agency Investigation,-1
New Construction,-1
New Ownership,-1
New Ownership - Followup,-1


In [281]:
grader.check("q1a")

q1a results: All test cases passed!

<br/>

---

## Question 1b


Given the variability of `ins['score']` observed in 1a, let's examine the inspection scores `ins['score']` further.

In [282]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

<center> <img src="pics/1b.png" width="400"/> 


In [288]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_group = (
    ins.groupby(["type", "Missing Score"])
    .size()
    .reset_index(name="Count")
)
ins_missing_score_group = ins_missing_score_group.set_index(["type", "Missing Score"])
ins_missing_score_group

,,Count
type,Missing Score,
Administrative or Document Review,True,4
Community Health Assessment,True,1
Complaint,True,1458
Complaint Reinspection/Followup,True,227
Foodborne Illness Investigation,True,115
Multi-agency Investigation,True,3
New Construction,True,994
New Ownership,True,1592
New Ownership - Followup,True,499


In [289]:
grader.check("q1b")

q1b results: All test cases passed!

<br/>

---

## Question 1c


Using `.groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the `pivot_table` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<table border="1" class="dataframe" >  <thead>    
    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr align="right">      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    
    <tr  align="right">      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    
    <tr  align="right">      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    
    <tr  align="right">      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    
    <tr  align="right">      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    
    <tr  align="right">      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    
    <tr  align="right">      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    
    <tr  align="right">      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    
    <tr  align="right">      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    
    <tr  align="right">      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    
    <tr  align="right">      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    
    <tr  align="right">      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    
    <tr  align="right">      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    
    <tr  align="right">      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>


In [155]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_pivot = ins.pivot_table(
    index = "type",
    columns = "Missing Score",
    values = "score",
    aggfunc = np.size,
    fill_value = (0)
)
ins_missing_score_pivot["Total"] = ins_missing_score_pivot["False"] + ins_missing_score_pivot["True"]
ins_missing_score_pivot = ins_missing_score_pivot.sort_values(["Total"], ascending=False)
ins_missing_score_pivot

Missing Score,False,True,Total
type,,,
Routine - Unscheduled,14031,46,14077
Reinspection/Followup,0,6439,6439
New Ownership,0,1592,1592
Complaint,0,1458,1458
New Construction,0,994,994
Non-inspection site visit,0,811,811
New Ownership - Followup,0,499,499
Structural Inspection,0,394,394
Complaint Reinspection/Followup,0,227,227


In [156]:
grader.check("q1c")

q1c results: All test cases passed!

Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/>

---

# Question 2: Joining Data Across Tables

In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

<br/>

--- 

## Question 2a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework. You might consider exploring this on your own. 

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [157]:
ins = ins[ins["score"] > 0]

We'll start by creating a new `DataFrame` called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` `DataFrame`. 

**Hint**: Use the `merge` method to join the `ins` `DataFrame` with the appropriate portion of the `bus` `DataFrame`. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`. The first few rows of the resulting `DataFrame` you create are shown below:

<img src="pics/2a.png" width="1080"/>

In [158]:
bus.head()

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124


In [104]:
del ins_named

In [123]:
ins["iid_base"] = ins["iid"].str.split("_").str[0]
bus["bid"] = bus["bid"].astype(str)
ins_named = ins.merge(bus, left_on = "iid_base", right_on = "bid")
ins_named = ins_named.drop(columns=["bid_y", "iid_base","city" , "state", "postal_code", "latitude", "longitude", "phone_number", "postal5"])
ins_named = ins_named.rename(columns={"bid_x":"bid"})
ins_named.head()

/tmp/ipykernel_116/3065726559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins["iid_base"] = ins["iid"].str.split("_").str[0]


,iid,date,score,type,timestamp,bid,Missing Score,name,address
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False,AMICI'S EAST COAST PIZZERIA,475 06th St
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False,UNCLE LEE CAFE,3608 BALBOA ST
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr


In [159]:
grader.check("q2a")

q2a results: All test cases passed!

<br/>

--- 

## Question 2b

Let's look at the 20 businesses in `ins_named` with the lowest **median** score. Order your results by the median score followed by the business name to break ties. The resulting table should look like the table below.

This one is pretty challenging! Don't forget to rename the `score` column. 

**Hint**: The `agg` function can accept a dictionary as an input. See the [agg documentation](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html). Additionally, when thinking about what aggregation functions to use, ask yourself what value would be in the `"name"` column for each entry across the group? Can we select just one of these values to represent the whole group?

As usual, **YOU SHOULD NOT USE LOOPS OR LIST COMPREHENSIONS**. Try and break down the problem piece by piece instead, gradually chaining together different `pandas` functions. Feel free to use more than one line!

<table border="1" class="dataframe">  <thead>    
    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>median score</th>    </tr> 
    <tr  align="right">  <th align="right">bid</th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    
    <tr  align="right">      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    
    <tr  align="right">        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    
    <tr  align="right">         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    
    <tr  align="right">          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    
    <tr  align="right">         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  
    <tr  align="right">          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   
    <tr  align="right">     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  
    <tr  align="right">         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  
    <tr  align="right">          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     
    <tr  align="right">        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    
    <tr  align="right">           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    
    <tr  align="right">         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    
    <tr  align="right">        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     
    <tr  align="right">          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr  align="right">          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> 
    <tr  align="right">         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>           
    <tr  align="right">        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>   
    <tr  align="right">         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    
    <tr  align="right">          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    
    <tr  align="right">        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>

In [161]:
group_scoring = (
    ins_named
    .groupby(["bid","name"])["score"]
    .agg(np.median)
    .reset_index("name")
)
# DO NOT USE LIST COMPREHENSIONS OR LOOPS OF ANY KIND!!!
twenty_lowest_scoring = group_scoring.sort_values("score", ascending=True).iloc[0:20,:]
twenty_lowest_scoring

,name,score
bid,,
84590,Chaat Corner,54.0
90622,Taqueria Lolita,57.0
94351,VBowls LLC,58.0
69282,New Jumbo Seafood Restaurant,60.5
1154,SUNFLOWER RESTAURANT,63.5
39776,Duc Loi Supermarket,64.0
69397,Minna SF Group LLC,64.0
93150,Chez Beesen,64.0
93502,Smoky Man,64.0


In [162]:
grader.check("q2b")

q2b results: All test cases passed!

<br/>

--- 
## Question 2c

Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. We should not be considering restaurants with missing scores, so this should not be a retaurant that has a score of `-1`. For fun: Look up the reviews for this restaurant on Yelp. Do you see any reviews that indicate this restaurant had health inspection issues?


In [166]:
worst_restaurant = ins_named.sort_values("score", ascending=True).iloc[0,7]
worst_restaurant

'Lollipot'

In [167]:
grader.check("q2c")

q2c results: All test cases passed!

<br/>

--- 
## Question 2d

Did this restaurant clean up its act? Look in the database to see if it scored better on its next inspection. Assign `cleaned_up` to `True` or `False`, depending on whether it performed better or not.

In [183]:
# SCRATCH WORK
check_cleaning = ins_named.sort_values("score", ascending=True)
check_cleaning = check_cleaning.set_index("name")
check_cleaning.loc["Lollipot", :]

,iid,date,score,type,timestamp,bid,Missing Score,address,lowercase_name
name,,,,,,,,,
Lollipot,86718_20180522,05/22/2018 12:00:00 AM,45,Routine - Unscheduled,2018-05-22,86718,False,890 Taraval St,lollipot
Lollipot,86718_20181005,10/05/2018 12:00:00 AM,90,Routine - Unscheduled,2018-10-05,86718,False,890 Taraval St,lollipot
Lollipot,86718_20161116,11/16/2016 12:00:00 AM,90,Routine - Unscheduled,2016-11-16,86718,False,890 Taraval St,lollipot


In [176]:
cleaned_up = False

In [177]:
grader.check("q2d")

q2d results: All test cases passed!

<br/><br/>

---
# Question 3: Let Them Eat Cake! 

Now that you've analyzed and found out which restaurants to avoid in SF, we can turn toward the more interesting question of what dessert places are the best! For the purposes of this question, we assume that cake is the best dessert (and rightfully so!)

<br/>

--- 
## Question 3a

In your quest to find the best cake shop, the first step is to find all the businesses in `ins_named` that **contain the word 'cake'** in their `name`, and assign the resulting `DataFrame` to `cake_shops`. To help you out, we created the `lowercase_name` column so you do not need to worry about checking for capitalized letters when checking if `name` contains `'cake'`.

**Hint:** You might find the `.str` accessors useful yet again!

In [179]:
ins_named['lowercase_name'] = ins_named['name'].str.lower()
cake_shops = ins_named[ins_named["lowercase_name"].str.contains("cake", na=False)]
cake_shops.head(10)

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
1637,26340_20170424,04/24/2017 12:00:00 AM,75,Routine - Unscheduled,2017-04-24,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1638,26340_20180514,05/14/2018 12:00:00 AM,85,Routine - Unscheduled,2018-05-14,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1911,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1912,2898_20180412,04/12/2018 12:00:00 AM,94,Routine - Unscheduled,2018-04-12,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1945,29304_20180112,01/12/2018 12:00:00 AM,92,Routine - Unscheduled,2018-01-12,29304,False,Kara's Cupcakes,3249 SCOTT,kara's cupcakes
2172,3210_20170110,01/10/2017 12:00:00 AM,96,Routine - Unscheduled,2017-01-10,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2173,3210_20170809,08/09/2017 12:00:00 AM,98,Routine - Unscheduled,2017-08-09,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2174,3210_20180205,02/05/2018 12:00:00 AM,96,Routine - Unscheduled,2018-02-05,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2175,3210_20181106,11/06/2018 12:00:00 AM,98,Routine - Unscheduled,2018-11-06,3210,False,International House of Pancakes,200 Beach St,international house of pancakes


In [180]:
grader.check("q3a")

q3a results: All test cases passed!

<br/>

--- 
## Question 3b

Assign `cake_at_least_3` to a `DataFrame` consisting of only those cake shops that have had at least 3 inspections. Remember, the `bid` uniquely defines a cake shop, not its `name`!

In [197]:
series_at_least_3 = cake_shops.groupby("bid")["type"].size()
series_at_least_3 = valid_bids[valid_bids >= 3]
cake_at_least_3 = cake_shops[cake_shops["bid"].isin(valid_bids.index)]
cake_at_least_3.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
1911,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1912,2898_20180412,04/12/2018 12:00:00 AM,94,Routine - Unscheduled,2018-04-12,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
2172,3210_20170110,01/10/2017 12:00:00 AM,96,Routine - Unscheduled,2017-01-10,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2173,3210_20170809,08/09/2017 12:00:00 AM,98,Routine - Unscheduled,2017-08-09,3210,False,International House of Pancakes,200 Beach St,international house of pancakes


In [198]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/>

---
## Question 3c

In the cell below, run the following line of code: `q3c_df = cake_at_least_3.sort_values('timestamp').groupby('bid').agg('first')`

Is the granularity of `cake_at_least_3` the same as the granularity of `q3c_df`? In other words, what does a single row of `q3c_df` represent, and what does a single row in `cake_at_least_3` represent? Explain the granularity of each `DataFrame`. Your answer does not need to be more than 2-3 lines, but you should be specific. 

**Note**: For more details on what the granularity of a `DataFrame` means, feel free to check [Section 5.2.1](https://ds100.org/course-notes/eda/eda.html#granularity) in the course notes! 

In [200]:
q3c_df = cake_at_least_3.sort_values('timestamp').groupby('bid').agg('first')
q3c_df.head()

,iid,date,score,type,timestamp,Missing Score,name,address,lowercase_name
bid,,,,,,,,,
2898,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
3210,3210_20170110,01/10/2017 12:00:00 AM,96,Routine - Unscheduled,2017-01-10,False,International House of Pancakes,200 Beach St,international house of pancakes
4630,4630_20170304,03/04/2017 12:00:00 AM,96,Routine - Unscheduled,2017-03-04,False,Zanze's Cheesecake,2405 Ocean Ave,zanze's cheesecake
7757,7757_20170131,01/31/2017 12:00:00 AM,92,Routine - Unscheduled,2017-01-31,False,CAKE GALLERY,290 09th St,cake gallery
7761,7761_20170523,05/23/2017 12:00:00 AM,86,Routine - Unscheduled,2017-05-23,False,LELENITA'S CAKES,3743 MISSION St,lelenita's cakes


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/>

---
## Question 3d

Rather than the inspection scores, you find that the number of vowels present in the business `name` is a better indicator of how good the cake is when it comes to the shops in `cake_at_least_3`. Using the helper function `count_vowels` we have defined for you, sort all the cake shops in `cake_at_least_3` based on the number of vowels in the business's name in descending order. Then, return a **Python `list`** consisting of the top 3 **uniquely named** cake shops using this sorted `DataFrame`. You should break ties using alphabetical ordering. You do not need to stick to the skeleton code provided, but you are **not allowed to add any new columns!**

This is pretty challenging, but rest assured, the price of knowing the best cake shops is well worth it! 

**Hint**: When working on this problem, it might be very helpful to carefully read the entirety of [Section 4.1](https://ds100.org/course-notes/pandas_3/pandas_3.html) in the course notes which touches on custom sorts! In particular, you may find Approaches 2 and 3 to be useful.

In [201]:
def count_vowels(name):
    vowels = 'aeiouAEIOU'
    return sum([letter in vowels for letter in name])

In [ ]:
(sorted_by_vowel_count["name"].sort_values(ascending=True).unique()[0:3]).astype(list)

In [253]:
cake_at_least_3["sorter"] = cake_at_least_3["name"].map(count_vowels)
sorted_by_vowel_count = cake_at_least_3.sort_values(by="sorter", ascending=False).drop(columns=["sorter"])
cake_at_least_3 = cake_at_least_3.drop(columns="sorter")
top_cakes = (sorted_by_vowel_count["name"].unique()[0:3]).tolist()
top_cakes

['International House of Pancakes',
 'The Cheesecake Factory',
 'SusieCakes Holdings, Inc.']

In [255]:
grader.check("q3d")

q3d results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/>

---
## Question 3e

Finally, to examine different parts of a chained `pandas` statement, describe the purpose of each of the functions used (`.loc`, `.groupby`, `idxmax()`) in words. 

Secondly, share what you think this line of code accomplishes. In other words, write a question that could be answered using this statement.

While the first part of this question will be graded for correctness, the second part of this question is a bit more open-ended. Answers demonstrating your understanding will get full credit.

In [256]:
cake_at_least_3.loc[cake_at_least_3.groupby("bid")["score"].idxmax()].head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
2173,3210_20170809,08/09/2017 12:00:00 AM,98,Routine - Unscheduled,2017-08-09,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
3675,4630_20180629,06/29/2018 12:00:00 AM,100,Routine - Unscheduled,2018-06-29,4630,False,Zanze's Cheesecake,2405 Ocean Ave,zanze's cheesecake
8510,7757_20190829,08/29/2019 12:00:00 AM,94,Routine - Unscheduled,2019-08-29,7757,False,CAKE GALLERY,290 09th St,cake gallery
8520,7761_20190319,03/19/2019 12:00:00 AM,92,Routine - Unscheduled,2019-03-19,7761,False,LELENITA'S CAKES,3743 MISSION St,lelenita's cakes


_Type your answer here, replacing this text._

In [ ]:
# You may do some scratch work in this cell, however, only your written answer will be graded. 
# Any outputs or dataframes you generate here will not be counted as part of your explanation.

<!-- END QUESTION -->

<br/><br/>

---

# Question 4: Restaurant Ratings Over Time

As a final challenge, we consider a scenario involving restaurants with multiple ratings over time.

Let's see which restaurant location has had the most extreme improvement in its scores. Let the "swing" of a restaurant location be defined as the difference between its highest-ever and lowest-ever score. **Only consider restaurant locations with at least 3 scores—that is, restaurants that were rated at least 3 times.** Assign `max_swing` to the name of the restaurant that has the maximum swing. 

We have not provided any skeleton, as there are many paths to getting the correct answer. The recommended approach to solving this problem is to break it down into smaller chunks (e.g., first, ensure all restaurants have at least 3 scores; second, compute the swing, etc.). This will likely require more than one line, so feel free to add/remove columns and define new temporary variables. Remember to assign your solution - a string containing the `name` of the restaurant location that experienced the most extreme improvement - to `max_swing` after you do so. 

**Note**: The "swing" is of a specific restaurant location. There might be some restaurants with multiple locations; we are focusing on the swing of a particular restaurant as specified by its `name` and `address`.

In [273]:
ins_at_least_3 = ins_named.groupby(["bid"])["type"].size()
ins_at_least_3 = ins_at_least_3[ins_at_least_3 >= 3]
res_at_least_3 = ins_named[ins_named["bid"].isin(ins_at_least_3.index)]
evolution_by_bid = (
    res_at_least_3
    .groupby("name")["score"]
    .agg(['min', 'max'])
    .reset_index("name")
)
evolution_by_bid["evolution"] = evolution_by_bid["max"] - evolution_by_bid["min"]
evolution_by_bid = evolution_by_bid.sort_values("evolution", ascending=False)
max_swing = evolution_by_bid.iloc[0,0]
max_swing

'Lollipot'

In [272]:
grader.check("q4")

q4 results: All test cases passed!

## Summary of Inspections Data

We have done a lot in this homework! 
 
- Broke down the inspection scores in detail using `.groupby` and `pivot_table`
- Joined the business and inspection data and identified the name of the restaurant with the worst rating
- Took a deep dive into the sweet world of cake and found the best spots under varying metrics
- Took a swing at analyzing how restaurant inspection scores change over time!

Over the course of this 2-part homework, we hope you have become more familiar with `pandas` - in terms of identifying when to use particular functions, how they work, when they can support EDA - as well as with EDA and Data Cleaning, as part of the broader Data Science Lifecycle. These tools will serve you well as a data scientist!

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Homework 2B! ##

Opal, Olive, and Brownie say hi :)

<img src = "pics/cats.jpg" width = "200">

### Course Content Feedback

If you have any feedback about this assignment or about any of our other weekly, weekly assignments, lectures, or discussions, please fill out the [Course Content Feedback Form](https://docs.google.com/forms/d/e/1FAIpQLSdpKA_E3b7PGqKSRqBUgSebb9bVFhRwRBv1ueisGsnKFkZkYg/viewform?usp=sf_link). Your input is valuable in helping us improve the quality and relevance of our content to better meet your needs and expectations!

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Once you submit this file to the HW 2B Coding assignment on Gradescope, Gradescope will automatically submit a PDF file with your written answers to the HW 2B Written assignment. If you run into any issues when running this cell, feel free to check this [section](https://ds100.org/debugging-guide/autograder_gradescope/autograder_gradescope.html#why-does-grader.exportrun_teststrue-fail-if-all-previous-tests-passed) in the Data 100 Debugging Guide.

**Important**: Please check that your written responses were generated and submitted correctly to the HW 2B Written Assignment.

**You are responsible for ensuring your submission follows our requirements and that the PDF for HW 2B written answers was generated/submitted correctly. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline. 

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)